In [1]:
import pandas as pd
from math import sqrt
import numpy as np

In [4]:
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [14]:
userInput = [{'title':'A Star Is Born (2018)', 'rating':3.5},
             {'title':'Jumanji: Welcome to the Jungle (2017)', 'rating':4},
             {'title':'Batman (1943)', 'rating':4.5},
             {'title':'Inception (2010)', 'rating':5},
             {'title':'La La Land (2016)', 'rating':4}]
inputMovies = pd.DataFrame(userInput)
print(inputMovies)

                                   title  rating
0                  A Star Is Born (2018)     3.5
1  Jumanji: Welcome to the Jungle (2017)     4.0
2                          Batman (1943)     4.5
3                       Inception (2010)     5.0
4                      La La Land (2016)     4.0


In [15]:
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', 1)
inputMovies = inputMovies[['movieId','title','rating']]
print(inputMovies)

   movieId                                  title  rating
0    79132                       Inception (2010)     5.0
1    91054                          Batman (1943)     4.5
2   164909                      La La Land (2016)     4.0
3   179401  Jumanji: Welcome to the Jungle (2017)     4.0
4   192385                  A Star Is Born (2018)     3.5


<ipython-input-15-b234746ee70f>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1)


In [18]:
userSubset = ratings[ratings['movieId'].isin(inputMovies['movieId'].tolist())]
print(userSubset.groupby('movieId').count())

         userId  rating  timestamp
movieId                           
79132      1079    1079       1079
164909       95      95         95
179401       51      51         51
192385       28      28         28


In [19]:
userSubsetGroup = userSubset.groupby(['userId'])

def take_5_elem(x):
    return len(x[1])

userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])

[(997,         userId  movieId  rating     timestamp
142065     997    79132     4.5  1.529242e+09
142769     997   164909     5.0  1.529249e+09
142949     997   179401     4.0  1.529250e+09
143041     997   192385     4.0  1.573856e+09), (1920,         userId  movieId  rating     timestamp
275858    1920    79132     5.0  1.425579e+09
276230    1920   164909     4.0  1.551670e+09
276281    1920   179401     3.0  1.551670e+09
276334    1920   192385     4.0  1.551670e+09), (3063,         userId  movieId  rating     timestamp
446367    3063    79132     4.5  1.487175e+09
446467    3063   164909     4.5  1.498424e+09
446491    3063   179401     2.5  1.528159e+09
446503    3063   192385     4.0  1.550500e+09), (3870,         userId  movieId  rating     timestamp
567925    3870    79132     4.0  1.291219e+09
568254    3870   164909     5.0  1.484561e+09
568298    3870   179401     4.0  1.545416e+09
568327    3870   192385     4.0  1.545416e+09), (4235,         userId  movieId  rating     t

<ipython-input-19-af8a4a42e498>:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)


In [20]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:

    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    nRatings = len(group)
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    tempRatingList = temp_df['rating'].tolist()


    tempGroupList = group['rating'].tolist()


    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [21]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  userId
0         0.345857     997
1         0.648886    1920
2         0.314870    3063
3        -0.132453    3870
4        -0.345857    4235


In [22]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  userId
19              1.0    4246
99              1.0    3652
79              1.0    2671
77              1.0    2457
73              1.0    2403


In [24]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
print(topUsersRating.head(100))

    similarityIndex  userId  movieId  rating     timestamp
0               1.0    4246        1     4.0  1.567305e+09
1               1.0    4246        2     4.0  1.569552e+09
2               1.0    4246        6     4.5  1.567305e+09
3               1.0    4246       32     4.5  1.567305e+09
4               1.0    4246       44     3.5  1.569552e+09
..              ...     ...      ...     ...           ...
95              1.0    4246     1222     5.0  1.567305e+09
96              1.0    4246     1225     5.0  1.567305e+09
97              1.0    4246     1230     4.5  1.567305e+09
98              1.0    4246     1232     3.5  1.569552e+09
99              1.0    4246     1234     4.0  1.567305e+09

[100 rows x 5 columns]


In [25]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

   similarityIndex  userId  movieId  rating     timestamp  weightedRating
0              1.0    4246        1     4.0  1.567305e+09             4.0
1              1.0    4246        2     4.0  1.569552e+09             4.0
2              1.0    4246        6     4.5  1.567305e+09             4.5
3              1.0    4246       32     4.5  1.567305e+09             4.5
4              1.0    4246       44     3.5  1.569552e+09             3.5


In [26]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

         sum_similarityIndex  sum_weightedRating
movieId                                         
1                  25.072483           92.846447
2                  16.304788           58.140565
3                   2.000000            6.000000
4                   1.000000            1.000000
5                   1.927173            5.781518


In [27]:
recommendation_df = pd.DataFrame()

#the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
print(recommendation_df.head(10))

         weighted average recommendation score  movieId
movieId                                                
1                                     3.703121        1
2                                     3.565858        2
3                                     3.000000        3
4                                     1.000000        4
5                                     3.000000        5
6                                     4.025711        6
7                                     2.500000        7
10                                    3.415583       10
11                                    3.734875       11
12                                    1.750000       12


In [28]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df)

         weighted average recommendation score  movieId
movieId                                                
92391                                      5.0    92391
1880                                       5.0     1880
160990                                     5.0   160990
37855                                      5.0    37855
96966                                      5.0    96966
...                                        ...      ...
157365                                     0.5   157365
233                                        0.5      233
2072                                       0.5     2072
135765                                     0.5   135765
151987                                     0.5   151987

[7877 rows x 2 columns]


In [30]:
recommended_movie=movies.loc[movies['movieId'].isin(recommendation_df['movieId'])]

recommended_movie=recommended_movie.loc[~recommended_movie.movieId.isin(userSubset['movieId'])]

print(recommended_movie)

       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
61935   207309                    Fractured (2019)   
61960   207367              Little Monsters (2019)   
62106   207890                    Countdown (2019)   
62126   208002                The Kill Team (2019)   
62149   208080                        Bruja (2019)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                              